# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 9:41AM,242404,16,SHL-8304088,"SHL Pharma, LLC",Released
1,Jul 21 2022 9:33AM,242403,19,8303574,"Omnivium Pharmaceuticals, LLC DBA WDSrx",Released
2,Jul 21 2022 9:24AM,242401,20,ALU0007354,Alumier Labs Inc.,Released
3,Jul 21 2022 9:24AM,242401,20,ALUR259007850,Alumier Labs Inc.,Released
4,Jul 21 2022 9:24AM,242401,20,ALUR601322965,Alumier Labs Inc.,Released
5,Jul 21 2022 9:24AM,242401,20,ALUR803859414,Alumier Labs Inc.,Released
6,Jul 21 2022 9:24AM,242401,20,ALUR827903137,Alumier Labs Inc.,Released
7,Jul 21 2022 9:24AM,242401,20,ALUR977513528,Alumier Labs Inc.,Released
8,Jul 21 2022 9:22AM,242399,16,SSF-8303569,Sartorius Stedim Filters Inc.,Released
9,Jul 21 2022 9:08AM,242396,10,8300961.1,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,242396,Released,1
32,242399,Released,1
33,242401,Released,6
34,242403,Released,1
35,242404,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242396,NaN,NaN,1.0
242399,NaN,NaN,1.0
242401,NaN,NaN,6.0
242403,NaN,NaN,1.0
242404,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242305,0.0,0.0,1.0
242306,0.0,0.0,2.0
242308,0.0,0.0,1.0
242319,0.0,0.0,1.0
242325,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242305,0,0,1
242306,0,0,2
242308,0,0,1
242319,0,0,1
242325,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242305,0,0,1
1,242306,0,0,2
2,242308,0,0,1
3,242319,0,0,1
4,242325,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242305,,,1
1,242306,,,2
2,242308,,,1
3,242319,,,1
4,242325,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC"
1,Jul 21 2022 9:33AM,242403,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx"
2,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.
8,Jul 21 2022 9:22AM,242399,16,Sartorius Stedim Filters Inc.
9,Jul 21 2022 9:08AM,242396,10,"Citieffe, Inc."
10,Jul 21 2022 9:07AM,242393,12,Hush Hush
11,Jul 21 2022 9:06AM,242395,10,"Nextsource Biotechnology, LLC"
17,Jul 21 2022 9:03AM,242387,10,ISDIN Corporation
52,Jul 21 2022 9:01AM,242391,10,Yusen – 3D Matrix
53,Jul 21 2022 8:57AM,242390,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",,,1
1,Jul 21 2022 9:33AM,242403,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",,,1
2,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,,,6
3,Jul 21 2022 9:22AM,242399,16,Sartorius Stedim Filters Inc.,,,1
4,Jul 21 2022 9:08AM,242396,10,"Citieffe, Inc.",,,1
5,Jul 21 2022 9:07AM,242393,12,Hush Hush,,,1
6,Jul 21 2022 9:06AM,242395,10,"Nextsource Biotechnology, LLC",,,6
7,Jul 21 2022 9:03AM,242387,10,ISDIN Corporation,,,35
8,Jul 21 2022 9:01AM,242391,10,Yusen – 3D Matrix,,,1
9,Jul 21 2022 8:57AM,242390,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",1,,
1,Jul 21 2022 9:33AM,242403,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1,,
2,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,6,,
3,Jul 21 2022 9:22AM,242399,16,Sartorius Stedim Filters Inc.,1,,
4,Jul 21 2022 9:08AM,242396,10,"Citieffe, Inc.",1,,
5,Jul 21 2022 9:07AM,242393,12,Hush Hush,1,,
6,Jul 21 2022 9:06AM,242395,10,"Nextsource Biotechnology, LLC",6,,
7,Jul 21 2022 9:03AM,242387,10,ISDIN Corporation,35,,
8,Jul 21 2022 9:01AM,242391,10,Yusen – 3D Matrix,1,,
9,Jul 21 2022 8:57AM,242390,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",1,,
1,Jul 21 2022 9:33AM,242403,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1,,
2,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,6,,
3,Jul 21 2022 9:22AM,242399,16,Sartorius Stedim Filters Inc.,1,,
4,Jul 21 2022 9:08AM,242396,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",1,NaN,NaN
1,Jul 21 2022 9:33AM,242403,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1,NaN,NaN
2,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,6,NaN,NaN
3,Jul 21 2022 9:22AM,242399,16,Sartorius Stedim Filters Inc.,1,NaN,NaN
4,Jul 21 2022 9:08AM,242396,10,"Citieffe, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 9:41AM,242404,16,"SHL Pharma, LLC",1,0.0,0.0
1,Jul 21 2022 9:33AM,242403,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1,0.0,0.0
2,Jul 21 2022 9:24AM,242401,20,Alumier Labs Inc.,6,0.0,0.0
3,Jul 21 2022 9:22AM,242399,16,Sartorius Stedim Filters Inc.,1,0.0,0.0
4,Jul 21 2022 9:08AM,242396,10,"Citieffe, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3635518,138,13.0,0.0
12,484729,2,0.0,0.0
16,484803,2,0.0,0.0
19,1211829,18,19.0,23.0
20,242401,6,0.0,0.0
21,1696490,7,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3635518,138,13.0,0.0
1,12,484729,2,0.0,0.0
2,16,484803,2,0.0,0.0
3,19,1211829,18,19.0,23.0
4,20,242401,6,0.0,0.0
5,21,1696490,7,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,138,13.0,0.0
1,12,2,0.0,0.0
2,16,2,0.0,0.0
3,19,18,19.0,23.0
4,20,6,0.0,0.0
5,21,7,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,138.0
1,12,Released,2.0
2,16,Released,2.0
3,19,Released,18.0
4,20,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,23.0,0.0,0.0
Executing,13.0,0.0,0.0,19.0,0.0,0.0
Released,138.0,2.0,2.0,18.0,6.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,13.0,0.0,0.0,19.0,0.0,0.0
2,Released,138.0,2.0,2.0,18.0,6.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,13.0,0.0,0.0,19.0,0.0,0.0
2,Released,138.0,2.0,2.0,18.0,6.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()